In [60]:
import numpy as np
from sympy import Symbol, diff

In [61]:
def create_F(number_of_functions, functions):
    F = np.array([[functions[i]] for i in range(number_of_functions)])
    return F

In [62]:
def create_J(number_of_functions, number_of_constants, functions):
    J = np.empty((number_of_functions, number_of_constants), dtype=object)
    for i in range(number_of_functions):
        for j in range(number_of_constants):
            J[i][j] = diff(functions[i], f"c{j+2}")
    return J

In [63]:
def update_c_dict(X, extra_dict={}):
    c_dict = {f"c{i+2}":X[i, 0] for i in range(len(X))}
    c_dict.update(extra_dict)
    return c_dict

In [64]:
def sub_F(F, c_dict):
    F = F.copy()
    for i in range(F.shape[0]):
        for j in range(F.shape[1]):
            F[i][j] = F[i][j].subs(c_dict)
    return F.astype(float)

In [65]:
def sub_J(J, c_dict):
    J = J.copy()
    for i in range(J.shape[0]):
        for j in range(J.shape[1]):
            J[i][j] = J[i][j].subs(c_dict)
    return J.astype(float)

In [66]:
def get_B(Y, B_k, delta_X):
    num = np.matmul((Y-np.matmul(B_k, delta_X)), delta_X.T)
    den = np.matmul(delta_X.T, delta_X)
    B = B_k + num/den
    return B.copy()

In [95]:
def calculate_nl_broyden(max_iter, max_tol, X0, theta_dict, F, J):
    
    X_list = []
    X_list.append(X0.copy())
    
    c_dict_0 = update_c_dict(X0, theta_dict)
    
    
    B_list = []
    B0 = sub_J(J, c_dict_0)
    B_list.append(B0.copy())
    
    F_list = []
    F0 = sub_F(F, c_dict_0)
    F_list.append(F0.copy())
    

    for k in range(1, max_iter+1):
        J_k = B_list[k-1]
        J_k_inv = np.linalg.inv(J_k)
#         print(J_k_inv)
#         print(F_list[k-1])
#         delta_X = np.dot(-J_k_inv, F_list[k-1])
        X_k = X_list[k-1] + delta_X
        X_list.append(X_k.copy())
        
        c_dict_k = update_c_dict(X_k, theta_dict)
        
        F_k = sub_F(F, c_dict_k)
#         print(F_k)
#         print(F_list[k-1])
        Y_k = F_k - F_list[k-1]
#         print("Y:")
#         display(Y_k)
        
        tolk = np.linalg.norm(delta_X)/np.linalg.norm(X_k)
#         print(tolk)
        if tolk < max_tol:
#             print(X_list)
            return X_k
        
        else:
            B_k = get_B(Y_k, J_k, delta_X)
            B_list.append(B_k.copy())
            
    raise Exception ("Não convergiu")

In [96]:
def run_broyden(max_iter, max_tol, X0, theta_dict):
    c2 = Symbol('c2')
    c3 = Symbol('c3')
    c4 = Symbol('c4')
    θ1 = Symbol('θ1')
    θ2 = Symbol('θ2')
    
    f1 = 2*c3**2+c2**2+6*c4**2-1
    f2 = 8*c3**3+6*c3*c2**2+36*c3*c2*c4+108*c3*c4**2-θ1
    f3 = 60*c3**4+60*c3**2*c2**2+576*c3**2*c2*c4+2232*c3**2*c4**2+252*c4**2*c2**2+1296*c4**3*c2+3348*c4**4+24*c2**3*c4+3*c2-θ2
        
#     f1 = c2+2*c3-2
#     f2 = c2**2+4*c3**2-4
    
    fns = [f1, f2, f3]
#     fns = [f1, f2]
    
#     number_of_functions = 2
#     number_of_constants = 2
    
    number_of_functions = 3
    number_of_constants = 3
    
    F = create_F(number_of_functions, fns)
    J = create_J(number_of_functions, number_of_constants, fns)

    X = calculate_nl_broyden(max_iter, max_tol, X0, theta_dict, F, J)
    return np.round(X.ravel(), 3).tolist()

In [97]:
max_iter = 100
max_tol = 1e-2
X0 = np.array([1, 0, 4]).reshape(-1, 1)
# X0 = np.array([2, 3]).reshape(-1, 1)

theta_dict = {"θ1": 0, "θ2": 6.5}

In [98]:
run_broyden(max_iter, max_tol, X0, theta_dict)

NameError: name 'delta_X' is not defined